In [1]:
# werkzeug
# shapely=1.7.1
# geopandas
# pandas
# contextily
# matplotlib
# docxtpl
# docx

In [1]:
import sys
sys.path.append('../_amigocloud')
from amigocloud import AmigoCloud

import requests
import docxtpl
from docxtpl import DocxTemplate
from docx.shared import Mm
from werkzeug.wrappers import Request, Response
from jinja2 import Template

from shapely import wkb
import geopandas
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

import collections
from datetime import datetime, timedelta
import qrcode
import os

import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)

def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_amigocloud(ruta_completa):
    try:
        with open(ruta_completa + r'\_keys\api_amigocloud_gral.key', 'r') as acceso:
            api_token = acceso.readline().strip()
        amigocloud = AmigoCloud(token=api_token)
        return amigocloud
    except FileNotFoundError:
        print("El archivo 'api_amigocloud.key' no se encuentra en la ruta especificada.")
    return None

In [3]:
def crear_cite(cod_canhero, nom_canhero, tipo_reporte):
    # Configura las credenciales
    scope = ["https://spreadsheets.google.com/feeds",
             'https://www.googleapis.com/auth/spreadsheets', 
             "https://www.googleapis.com/auth/drive.file", 
             "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(path_file_gapi, scope)
    client = gspread.authorize(creds)
    # Abre la hoja de cálculo con ID y la hoja específica
    spreadsheet = client.open_by_key("1Wbmmjy9s8JdXKP8qM_eA_OrCV--F1UZLnzQJ--ASOFI")
    sheet = spreadsheet.sheet1
    # Lee los datos
    data_sheet = sheet.get_all_records()
    df = pd.DataFrame(data_sheet)
    # Encuentra el número más grande y lo incrementamos y agrega un nuevo registro
    max_value = df['No'].max()
    no = int(max_value + 1)
    # llenamos los demas campos
    fecha = datetime.now().strftime("%d/%m/%Y")
    cod_ca = int(cod_canhero)
    nom_ca = nom_canhero
    tipo = tipo_reporte
    #agregar el registro
    sheet.append_row([no, fecha, cod_ca, nom_ca, tipo])
    return no

In [4]:
def generar_reporte(codigo_canhero):
    # EXTRAER DATOS DE CREDITO
    query_credito = {'query': 'select * from dataset_307560 where cast(SPLIT_PART(canhero, \'/\', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_credito)
    credito_data = credito_select['data'][0]
    credito_data['fecha_informe'] = (datetime.strptime(credito_data['fecha_2da_insp'].split(' ')[0], '%Y-%m-%d') + timedelta(days=1)).strftime('%d/%m/%Y')
    credito_data['fecha_2da_insp'] = datetime.strptime(credito_data['fecha_2da_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    
    cite = crear_cite(credito_data['canhero'].split(' / ')[0], credito_data['canhero'].split(' / ')[1], 'CRSI')
    credito_data['cite_2da_insp'] = cite
    
    if credito_data['obs_2da'] == None:
        credito_data['obs_2da'] = ''
    
    credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

    # EXTRAER DATOS DE PROPIEDADES
    query_propiedades = {'query': 'select * from dataset_307562 where insp_ref_prop = \'{insp_ref_prop}\''.format(insp_ref_prop=credito.amigo_id)}
    propiedades_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_propiedades)
    propiedades_data = propiedades_select['data']

    # EXTRAER DATOS DE LOTES
    propiedades = []
    for prop in propiedades_data:
        prop['lotes'] = []
        query_lotes = {'query': "select * from dataset_307563 where (estado_lote_2da = 'SEMBRADO' or estado_lote_2da = 'NO SEMBRADO') and prop_ref_lote=\'{prop_ref_lote}\'".format(prop_ref_lote=prop['amigo_id'])}
        lotes_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_lotes)
        for lote_select in lotes_select['data']:
            lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
            prop['lotes'].append(lote)
        if len(prop['lotes']) > 0:
            propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))

    # EXTRAER FOTOS
    fotos = []
    for propiedad in propiedades:
        for lote in propiedad.lotes:
            print(lote.lote, lote.amigo_id)
            query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_48961 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
            fotos_filename = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_fotos)
            if fotos_filename['data'] != []:
                dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
                fotos.append(dic)

    # GENERERAR PLANOS
    i = 0
    lista_planos = []
    for propiedad in propiedades:
        i = i + 1
        lista_planos.append(generar_plano(propiedad, i))
    
    
    # GENERAR QR
    texto = '''
--UTEA--
--2DA INSPECCION CREDITO SIEMBRA--

Codigo de Inspeccion: {cite}
Fecha de 2da Inspeccion: {fecha}
Nombre de Canero: {canero}
Codigo Canero: {cod_ca}
Total Solicitado: {soli}
Total Area Insp.: {total_area} ha
Estado de Insp.: {estado}
Tecnico Responsable: {tecnico}'''\
    .format(
        cite=str(credito.cite_2da_insp) + '-' + credito.fecha_2da_insp.split('/')[2], 
        fecha=credito.fecha_2da_insp, 
        canero=credito.canhero.split(' / ')[1],
        cod_ca=credito.canhero.split(' / ')[0],
        soli=credito.solicitado,
        total_area=credito.total_2da_insp,
        estado=credito.estado_2da_insp,
        tecnico=credito.respon_2da_insp)
    # Generar el código QR
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(texto)
    qr.make(fit=True)
    
    qr_img = qr.make_image(fill_color="black", back_color="white")
    
    cod_ca = credito.canhero.split(' / ')[0]
    nom_ca = credito.canhero.split(' / ')[1]
    filename_qr = "qr_" + cod_ca + '_CRS_' + credito.fecha_informe.replace('/', '-') + '_' + nom_ca
    ruta_qr = ruta_completa + '/planos/' + filename_qr + '.png'
    # Guardar el código QR en un archivo
    qr_img.save(ruta_qr)

    # GENERAR REPORTE
    doc = DocxTemplate(ruta_completa + "/templates/tpl_inf_credito_2da.docx")
    #generar lista de InlineImage de planos 
    lista_InlineImage = []
    for plano in lista_planos:
        lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        file = open(ruta_completa + '/fotos/' + foto['file_name'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta_completa+'/fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})
    firma_respon = None
    if credito.respon_2da_insp == 'Rogelio Acuña Rodríguez':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta_completa+'/templates/firma_rogelio.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta_completa+'/templates/firma_jaldin.png', width=Mm(60))
    # se crea obj inline par imagen QR
    qr_file = docxtpl.InlineImage(doc, image_descriptor=ruta_qr, width=Mm(30))
    
    context = {'credito':credito, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon, 'qr':qr_file}
    doc.render(context)
    cod_ca = credito.canhero.split(' / ')[0]
    nom_ca = credito.canhero.split(' / ')[1]
    file_name = "_" + cod_ca + '_CRS_' + credito.fecha_informe.replace('/', '-') + '_' + nom_ca + '_' + str(credito.id)
    doc.save(ruta_completa + '/informes/' + file_name + '.docx')

In [5]:
def generar_plano(propiedad, i):
    lotes_lista = []
    for lote in propiedad.lotes:
        lotes_lista.append(lote._asdict())
    df = pd.DataFrame(lotes_lista)
    df['geometria'] = df['geometria'].apply(convertir_wkb)

    #Convertir a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    #data
    #ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]

    data.crs = "EPSG:4326"
    data = data.to_crs(epsg=3857)

    
    fig = plt.figure(i, figsize=(20,20))
    #fig = plt.figure(1)
    ax = None
    ax = fig.add_subplot()

    #data.set_crs(epsg=4326, inplace=True)

    data.apply(lambda x: ax.annotate(text=x.lote + ' \n' + str(x.superficie) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);

    #data.boundary.plot(ax=ax, color=(0,0,0,0), linewidth=0)

    #data = data.to_crs(epsg=3857)

    minx, miny, maxx, maxy = data.total_bounds
    ax.set_xlim(minx - 500, maxx + 500)
    ax.set_ylim(miny - 400, maxy + 400)

    data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    #data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2)
    
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    ax.set_axis_off()
    ax.set_title(str(propiedad.propiedad), fontsize=20)
    path = ruta_completa + '/planos/' + str(propiedad.amigo_id) + '.jpeg'
    fig.savefig(path, dpi = 300, bbox_inches='tight')
    
    plt.clf()
    return path

In [6]:
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [7]:
path_file_gapi = unidad + r'\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON\_keys\client_secret_google_sheets.json'
path_file_gapi

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON\\_keys\\client_secret_google_sheets.json'

In [8]:
# Uso de la función
amigocloud = get_amigocloud(ruta_completa)
print(amigocloud)

In [9]:
semana = 'S19'
query = {'query': 'select split_part(canhero, \' / \', 1) cod, split_part(canhero, \' / \', 2) nom from dataset_307560 where semana_2da = \'{sem}\''.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/32672/sql'
select = amigocloud.get(project, query)
data = select['data']

In [10]:
data

[{'cod': '18062', 'nom': 'MERCADO MERCADO RUBEN DARIO'},
 {'cod': '8842', 'nom': 'MUÑOZ SUAREZ HUMBERTO'},
 {'cod': '767', 'nom': 'PROSER S.R.L'},
 {'cod': '40256', 'nom': 'INSERVA DAMIANA S.R.L'},
 {'cod': '12162', 'nom': 'ROCA REYES MARCOS'},
 {'cod': '41594', 'nom': 'AGROPECUARIA CAMPO DULCE S.R.L.'},
 {'cod': '31160', 'nom': 'PIZARRO PARADA HUGO ANDRES'},
 {'cod': '594', 'nom': 'AGROPECUARIA FLAMBOYAN'}]

In [11]:
#lista = [1068, 18062, 10739, 41955, 2392, 15162, 2517, 8706, 15085, 430, 874, 5498, 2505, 2274]
lista = [int(i['cod']) for i in data]

In [12]:
print(len(lista), 'informes')
print(lista)

8 informes
[18062, 8842, 767, 40256, 12162, 41594, 31160, 594]


In [13]:
for i in lista:
    generar_reporte(i)
    print('======================', i)

L14.1 c67f217e7c3c46c99684ada40de96cf0
L6.2 80a469aa833d4327b2ff254a04f04d4a
L6.1 F6FC96F80ADA4EE5A5A1EEE164248E4B
L9.2 52cf7e3f02ba462b84f0f6a64cc4ef5c
====================== 18062
L2 cc52541f8c0a4d11986ae9af3baf0986
====================== 8842
L1 63a432cfc4ae4f0da639f3392949375b
L3.1 785b82f35f784c669f5d7ecbcbef9129
L1 aa331a85b1b94ae1b5bbda66ba5e9146
A2 fe316c1a804d4156bd1205da9b95557c
====================== 767
L20.2 709d0f87fffa41ad96d91c74b9e5390f
L1.1 91fb30c0f8c14893be174f75b6b75c41
L16.4 dcba2f04c6914c5f9c956fae2c1fd14d
L9.2.2 0f626254b74a4e92b3a48b4f34dcbef7
L17.2 2e9506c96e2f44988845e132c51c2e18
L23.3 dfcde4b1563041379aba76f276a3b484
L2.3 a4d7e444af28420ab66e5d56cb20d5c2
L26.3 5c471c10ce534cc3ba10e3cca4368a63
L6.3 f379c22ab0164594811f5303d3071258
L22.3 6614fa10ce4849dda1062855fe37ad66
L3 f988e1d875304a1d9fadd1f3eef7a5f8
L1.1 c67c4f6255a94bb99b0c76334c54c81d
====================== 40256
L13.4 3a10e3ee5991478ca83b162a46098be9
L11 db87bea247554fcc9b2d30c9e651f083
L15.1 d2be82a2

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>